In [22]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [23]:
import re

def preprocess_input(input):
    regexp = r'[\\\/\.,;:()]+\w*'
    regexp2 = r'\b(?<![\\\/\.\,\\:\;(\)\+\-])\w+\b'
    regexp3 = r'\b(?<![\\\/])\w+\.*\w*\b'
    # Remove html tags
    res = BeautifulSoup(input, 'lxml').get_text()

    # Remove words that are preceded by a special character along with special characters
    #result = re.sub(regexp2, '', res)

    # Save words that are not preceded by a '\' or '/' and that can be followed by '.se' for example
    result = re.findall(regexp2, res)

    # Remove excessive whitespaces
    result = " ".join(result)
    #result = " ".join(result.split())
   
    return result

In [24]:
from os.path import exists

if(not exists('preprocessed_articles.csv')):  
    df_articles = pd.read_csv('sävsjö_articles.csv')
    df_articles = df_articles.dropna()
    # columns: ['ID', 'Title', 'Text']
    # iterate over 'Title' and 'Text' columns and preprocess the texts
    for col in df_articles.columns[1:]:
        df_articles[col] = df_articles[col].astype(str)
        df_articles[col] = df_articles[col].apply(lambda x: preprocess_input(x))
    
else:
    df_articles = pd.read_csv('preprocessed_articles.csv')

c:\Users\strep\miniconda3\envs\nlp\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [27]:
df_articles.head()

,ID,Title,Text,Entities,Labels
0,2,Aktiespararna säger ja till Gunvorbudet di,Aktiespararna rekommenderar sina medlemmar att...,"['Gunvor Group', 'Rörvik', 'Gunvorbudet di']","['PRS', 'ORG', 'ORG']"
2,7,Man fast i en timme i grop kuriren,En man i 60 föll ner i en grop som var grävd p...,['Sävsjö'],['LOC']
3,8,Mannen fastnade i en grop i flera timmar expre...,En man i 60 trillade ner i ett grävt hål på si...,['David'],['PRS']
4,56,En motorcykel,Här är den längre versionen,[],[]
5,66,Stulna braskaminer för 50 000 ska värma,Det börjar bli vinter och kallt Då är det skön...,"['Smålandsvillan', 'Vrigstad']","['ORG', 'LOC']"


In [26]:
#from spacy.pipeline.ner import EntityRecognizer
import spacy
from os.path import exists

if(not exists('preprocessed_articles.csv')):   

    nlp = spacy.load("sv_core_news_sm")
    df_articles['Entities'] = '0'
    df_articles['Labels'] = '0'

    for index, row in df_articles.iterrows():
        doc_text = nlp(row['Text'])
        doc_title = nlp(row['Title'])
    
        doc_ents = doc_text.ents + doc_title.ents
        unique_ents = list({keyword.__repr__(): keyword for keyword in doc_ents}.values())
        
        entities = [str(x) for x in unique_ents]
        labels = [str(labels.label_) for labels in unique_ents]
        
        df_articles.loc[index,['Entities', 'Labels']] = f'{entities}', f'{labels}'

    df_articles.to_csv('preprocessed_articles.csv')

df_articles.head(10)


,ID,Title,Text,Entities,Labels
0,2,Aktiespararna säger ja till Gunvorbudet di,Aktiespararna rekommenderar sina medlemmar att...,"['Gunvor Group', 'Rörvik', 'Gunvorbudet di']","['PRS', 'ORG', 'ORG']"
2,7,Man fast i en timme i grop kuriren,En man i 60 föll ner i en grop som var grävd p...,['Sävsjö'],['LOC']
3,8,Mannen fastnade i en grop i flera timmar expre...,En man i 60 trillade ner i ett grävt hål på si...,['David'],['PRS']
4,56,En motorcykel,Här är den längre versionen,[],[]
5,66,Stulna braskaminer för 50 000 ska värma,Det börjar bli vinter och kallt Då är det skön...,"['Smålandsvillan', 'Vrigstad']","['ORG', 'LOC']"
6,67,Sävsjö får ny skolchef,Sävsjö kommun ska få en ny skolchef Han heter ...,"['Stefan Claesson', 'Jönköping', 'Sävsjö']","['PRS', 'LOC', 'LOC']"
7,70,Krösatågen kan bli försenade,Ett signalfel på sträckan Södertälje på fredag...,"['Södertälje', 'Länstrafiken', 'i god tid']","['LOC', 'LOC', 'TME']"
8,72,Sävsjöpensionärer gör uppror,Stora grupper av pensionärer i Sävsjö är på kr...,"['Sävsjö', 'Vrigstad', 'SPF']","['LOC', 'LOC', 'ORG']"
9,78,Linus testar,Ett test jdklsa jfkldsa sjfkldsaö jfklsdaöjfls...,['Linus'],['PRS']
10,79,Linus testar 2,Testar 2,[],[]
